# Fiddler examples have moved! [Deprecation Notice]

Dear user thank you for using fiddler product, we appreciate your time! We have moved the examples to a new github repo located at the following link


***
# [New fiddler-examples repo](https://github.com/fiddler-labs/fiddler-examples)
***

# Fiddler-Snowflake Integration 1 - Add Model

This notebook is used in conjuction with the guide on how to use Fiddler for you machine learning model data stored in Snowflake. Link

In this notebook we will look at how we can use data stored in Snowflake to upload to fiddler as baseline data for your ML model. We will also add your model on the Fiddler platform.

[Link](https://colab.research.google.com/github/fiddler-labs/fiddler-samples/blob/master/content_root/tutorial/integration-examples/snowflake/Fiddler-Snowflake%20Integration%20-%20Event%20Publishing.ipynb) to part 2 of the guide on how to publish production data on Fiddler.

## Establish connection to Snowflake

Install required libraries to loading data from Snowflake

In [ ]:
!pip install snowflake

In [ ]:
# Imports
import pandas as pd
from snowflake import connector
import getpass

In [ ]:
# Enter your Snowflake credentials
snowflake_username = ''
snowflake_account = ''
snowflake_role = ''
snowflake_warehouse = ''

In [ ]:
# Enter your Snowflake password
snowflake_password = getpass.getpass("Enter password:")

In [ ]:
# establish Snowflake connection
connection = connector.connect(user=snowflake_username, 
                               password=snowflake_password, 
                               account=snowflake_account, 
                               role=snowflake_role, 
                               warehouse=snowflake_warehouse
                              )

Below we use the query to load the required data from Snowflake table

In [ ]:
# sample SQL query
sql_query = 'select * from FIDDLER.FIDDLER_SCHEMA.CHURN_BASELINE LIMIT 100'

In [ ]:
# create cursor object
cursor = connection.cursor()

# execute SQL query inside Snowflake
cursor.execute(sql_query)

store the query results as a pandas DataFrame

In [ ]:
baseline_df = cursor.fetch_pandas_all()

In [ ]:
# print dataset details
print('Dataset Dimensions:', baseline_df.shape)
print('Columns:', baseline_df.columns)
baseline_df.head()

## Add your model to Fiddler

### Imports

We will perform the following steps to get started on the Fiddler platform using the Snowflake data - 
1. Upload a baseline dataset
2. Add your model with Fiddler

If you are new to Fiddler and want a detailed explanation on how to setup the fiddler environment please refer to our Getting Started Guide.


In [ ]:
!pip install -q fiddler-client;

In [ ]:
import numpy as np
import pandas as pd
import fiddler as fdl

print(f"Running client version {fdl.__version__}")

### Connect to Fiddler

In [ ]:
URL = #
ORG_ID = #
AUTH_TOKEN = #

In [ ]:
# Initiate Fiddler client
client = fdl.FiddlerApi(
    url=URL,
    org_id=ORG_ID,
    auth_token=AUTH_TOKEN
)

In [ ]:
PROJECT_ID = 'churn_prediction_sf'

client.create_project(PROJECT_ID)

### Upload a baseline dataset

Now that we already have baseline data imported from Snowflake, we can go ahead and upload it.

In [ ]:
# create dataset info object to be used by 'upload_dataset'
dataset_info = fdl.DatasetInfo.from_dataframe(baseline_df, max_inferred_cardinality=100)
dataset_info

In [ ]:
# upload data to fiddler
DATASET_ID = 'churn_data'

client.upload_dataset(
    project_id=PROJECT_ID,
    dataset_id=DATASET_ID,
    dataset={
        'baseline': baseline_df
    },
    info=dataset_info
)

### Add your model

In [ ]:
# Specify task
model_task = 'binary'

if model_task == 'regression':
    model_task = fdl.ModelTask.REGRESSION
    
elif model_task == 'binary':
    model_task = fdl.ModelTask.BINARY_CLASSIFICATION

elif model_task == 'multiclass':
    model_task = fdl.ModelTask.MULTICLASS_CLASSIFICATION

    
# Specify column types
target = 'churn'
outputs = ['predicted_churn']
decision_cols = ['decision']
features = ['geography', 'gender', 'age', 'tenure', 'balance', 'numofproducts', 'hascrcard', 'isactivemember', 'estimatedsalary']
    
# Generate ModelInfo
model_info = fdl.ModelInfo.from_dataset_info(
    dataset_info=dataset_info,
    dataset_id=DATASET_ID,
    model_task=model_task,
    target=target,
    outputs=outputs,
    decision_cols=decision_cols,
    features=features
)
model_info

In [ ]:
MODEL_ID = 'churn_classifier'

client.add_model(
    project_id=PROJECT_ID,
    dataset_id=DATASET_ID,
    model_id=MODEL_ID,
    model_info=model_info
)


Now, that we have added the mode, we can go ahead and publish events to Fiddler platform.

To learn how to import data from Snowflake and publish to Fiddler, refer to our [notebook](https://colab.research.google.com/github/fiddler-labs/fiddler-samples/blob/master/content_root/tutorial/integration-examples/snowflake/Fiddler-Snowflake%20Integration%20-%20Event%20Publishing.ipynb) on Event Publishing.